<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/AtomGen_Carbon_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q pytorch-lightning wandb torchmetrics==0.6.0 pymatgen==2022.4.26
!pip install -q hydra-core jarvis-tools python-dotenv p-tqdm accelerate

2.1.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 42.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.4/329.4 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 93.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
os.chdir('/content')
if not os.path.exists('cdvae'):
  !git clone https://github.com/JARVIS-Materials-Design/cdvae.git

os.chdir('cdvae')
!pip install -e .

Cloning into 'cdvae'...
remote: Enumerating objects: 253, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 253 (delta 38), reused 52 (delta 32), pack-reused 164
Receiving objects: 100% (253/253), 146.69 MiB | 29.15 MiB/s, done.
Resolving deltas: 100% (90/90), done.
Updating files: 100% (99/99), done.
Obtaining file:///content/cdvae
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cdvae


In [3]:
import os
if not os.path.exists("/content/cdvae/WABDB"):
  os.makedirs("/content/cdvae/WABDB")
if not os.path.exists("/content/cdvae/HYDRA_JOBS"):
  os.makedirs("/content/cdvae/HYDRA_JOBS")
os.environ["PROJECT_ROOT"]="/content/cdvae"
os.environ["WABDB"]="/content/cdvae/WABDB"
os.environ["WABDB_DIR"]="/content/cdvae/WABDB"
os.environ["HYDRA_JOBS"]="/content/cdvae/HYDRA_JOBS"
%env HYDRA_FULL_ERROR=1


env: HYDRA_FULL_ERROR=1


In [4]:
!echo $HYDRA_FULL_ERROR

1


In [5]:

import yaml
import pprint

with open('/content/cdvae/conf/train/default.yaml','r') as f:
  yam = yaml.safe_load(f)
#yam.pop('early_stopping')
#yam['pl_trainer']['fast_dev_run']=True
yam['pl_trainer']['gpus']=0

with open('/content/cdvae/conf/train/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

with open('/content/cdvae/conf/optim/default.yaml','r') as f:
  yam = yaml.safe_load(f)

yam['use_lr_scheduler']=False

with open('/content/cdvae/conf/optim/default.yaml','w') as f:
  yaml.dump(yam,f)

pprint.pprint(yam)

{'deterministic': False,
 'early_stopping': {'patience': '${data.early_stopping_patience}',
                    'verbose': False},
 'model_checkpoints': {'save_top_k': 1, 'verbose': False},
 'monitor_metric': 'val_loss',
 'monitor_metric_mode': 'min',
 'pl_trainer': {'accumulate_grad_batches': 1,
                'fast_dev_run': False,
                'gpus': 0,
                'gradient_clip_algorithm': 'value',
                'gradient_clip_val': 0.5,
                'max_epochs': '${data.train_max_epochs}',
                'num_sanity_val_steps': 2,
                'precision': 32,
                'profiler': 'simple'},
 'random_seed': 42}
{'lr_scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
                  'factor': 0.6,
                  'min_lr': '1e-4',
                  'patience': 30},
 'optimizer': {'_target_': 'torch.optim.Adam',
               'betas': [0.9, 0.999],
               'eps': '1e-08',
               'lr': 0.001,
               'weight_de

In [ ]:
%%time
import os
os.environ["WANDB_ANONYMOUS"] = "must"
!python cdvae/run.py data=carbon expname=carbon_test model.predict_property=True

2024-01-30 11:58:23.199588: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-30 11:58:23.199639: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-30 11:58:23.200653: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-30 11:58:24.299608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/cdvae/cdvae/run.py:164: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path=str(PROJECT_ROOT / "conf"), con